Import requirements

In [4]:
import os
import pandas as pd
from datetime import datetime as dt
import unittest

# pd.options.mode.chained_assignment = None
pd.options.display.width=0
pd.set_option('display.expand_frame_repr', False)

Here, we create the CFB Model Dataframes

In [5]:
from CFBDataframe import CFBDataframe

Build the CFB dataframe with the scraped game data

In [6]:
cfb_df = CFBDataframe()
for directory in next(os.walk('../data'))[1]:
    # non time specific dfs
    if directory in ["cleaned_games","recruiting_groups", "recruiting_teams", "venues"]:
        try:
            cfb_df.csv_to_df([directory])
        except FileNotFoundError:
            print("Missing data from", directory)
    else:
        # season specific dfs
        for year in range(2010, dt.today().year):
            # week specific df
            if directory == "player_game_stats":
                for week in range(1, 17):
                    try:
                        cfb_df.csv_to_df([directory, week, year])
                    except FileNotFoundError:
                        print("Missing game data for", year, "week", week)
            elif directory == "__pycache__":
                continue
            else:
                try:
                    cfb_df.csv_to_df([directory, year])
                except FileNotFoundError:
                    print("Missing", year, "data for", directory)

Here, we initialize attributes and methods used to manipulate CFB Model

In [7]:
from CFBModel import CFBModel

Create and perform operations on model

In [8]:
cfb_model = CFBModel(cfb_df.data_map)
home_favor = cfb_model.home_favored()
print("Home teams win by an average of {:.2f} points".format(home_favor))
num_cells = sum(cfb_model.data[k].size for k in cfb_model.data)
print("We have {} cells of data to work with 👀".format(num_cells))
# print("Prediction", cfb_model.regression_predict([0.77, 34]))

In [9]:
spread_regression_model = CFBModel(cfb_df.data_map)

In [10]:
spread_model_score = spread_regression_model.spread_prediction()
print(spread_model_score)

[15:04:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
0.24035397058506502


Unit testing

In [11]:
class TestRanking(unittest.TestCase):

    def test_dfs(self):
        self.assertGreater(num_cells, 0)
        self.assertEqual(type(cfb_model.data), dict)
        self.assertEqual(len(cfb_model.data), 10)


unittest.main(argv=[''], verbosity=0, exit=False)

ERROR: test_dfs (__main__.TestRanking)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-11-c78273acaa5d>", line 4, in test_dfs
    self.assertGreater(num_cells, 0)
NameError: name 'num_cells' is not defined

----------------------------------------------------------------------
Ran 1 test in 0.000s

FAILED (errors=1)
